# Linear Regression

Objectif : Predict the house price in USA

In [1]:
import org.apache.spark.ml.regression.LinearRegression
import org.apache.spark.ml.feature.VectorAssembler
import org.apache.spark.ml.linalg.Vector

Intitializing Scala interpreter ...

Spark Web UI available at http://192.168.43.118:4043
SparkContext available as 'sc' (version = 2.3.0, master = local[*], app id = local-1546204496716)
SparkSession available as 'spark'


import org.apache.spark.ml.regression.LinearRegression
import org.apache.spark.ml.feature.VectorAssembler
import org.apache.spark.ml.linalg.Vector


In [2]:
val file ="Machine_Learning_Sections/Regression/USA_Housing.csv"
var df = spark.read.option("header","true").option("inferSchema","true").format("csv").load(file)

file: String = Machine_Learning_Sections/Regression/USA_Housing.csv
df: org.apache.spark.sql.DataFrame = [AvgAreaIncome: string, AvgAreaHouseAge: string ... 5 more fields]


In [4]:
df = df.na.drop()

df: org.apache.spark.sql.DataFrame = [AvgAreaIncome: string, AvgAreaHouseAge: string ... 5 more fields]


In [5]:
df.count

res0: Long = 5000


In [6]:
df.printSchema

root
 |-- AvgAreaIncome: string (nullable = true)
 |-- AvgAreaHouseAge: string (nullable = true)
 |-- AvgAreaNumberOfRooms: double (nullable = true)
 |-- AvgAreaNumberOfBedrooms: double (nullable = true)
 |-- AreaPopulation: double (nullable = true)
 |-- Price: double (nullable = true)
 |-- Address: string (nullable = true)



In [7]:
// change data type (string to double )
df = df.selectExpr("cast(AvgAreaIncome as double)",
                   "cast(AvgAreaHouseAge as double)",
                   "AvgAreaNumberOfRooms",
                   "AvgAreaNumberOfBedrooms",
                   "AreaPopulation",
                   "Price",
                   "Address")

df: org.apache.spark.sql.DataFrame = [AvgAreaIncome: double, AvgAreaHouseAge: double ... 5 more fields]


In [8]:
df.printSchema

root
 |-- AvgAreaIncome: double (nullable = true)
 |-- AvgAreaHouseAge: double (nullable = true)
 |-- AvgAreaNumberOfRooms: double (nullable = true)
 |-- AvgAreaNumberOfBedrooms: double (nullable = true)
 |-- AreaPopulation: double (nullable = true)
 |-- Price: double (nullable = true)
 |-- Address: string (nullable = true)



In [9]:
df.columns

res3: Array[String] = Array(AvgAreaIncome, AvgAreaHouseAge, AvgAreaNumberOfRooms, AvgAreaNumberOfBedrooms, AreaPopulation, Price, Address)


In [10]:
val df1 = df.select(df("Price").as("label"),
                    $"AvgAreaIncome", 
                    $"AvgAreaHouseAge", 
                    $"AvgAreaNumberOfRooms",
                    $"AvgAreaNumberOfBedrooms", 
                    $"AreaPopulation", $"Address")

df1: org.apache.spark.sql.DataFrame = [label: double, AvgAreaIncome: double ... 5 more fields]


In [11]:
val assembler = new VectorAssembler().setInputCols(Array("AvgAreaIncome",
		 "AvgAreaHouseAge",
		 "AvgAreaNumberOfRooms",
		 "AvgAreaNumberOfBedrooms",
		 "AreaPopulation")).setOutputCol("features")

assembler: org.apache.spark.ml.feature.VectorAssembler = vecAssembler_3365e7e6e68b


In [12]:
val output = assembler.transform(df1).select($"label",$"features")

output: org.apache.spark.sql.DataFrame = [label: double, features: vector]


In [13]:
val lr = new LinearRegression()
val lrModel = lr.fit(output)

2018-12-30 22:53:43 WARN  WeightedLeastSquares:66 - regParam is zero, which might cause numerical instability and overfitting.
2018-12-30 22:53:43 WARN  BLAS:61 - Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
2018-12-30 22:53:43 WARN  BLAS:61 - Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS
2018-12-30 22:53:44 WARN  LAPACK:61 - Failed to load implementation from: com.github.fommil.netlib.NativeSystemLAPACK
2018-12-30 22:53:44 WARN  LAPACK:61 - Failed to load implementation from: com.github.fommil.netlib.NativeRefLAPACK


lr: org.apache.spark.ml.regression.LinearRegression = linReg_3d3e3c0e4475
lrModel: org.apache.spark.ml.regression.LinearRegressionModel = linReg_3d3e3c0e4475


In [14]:
val Msummary = lrModel.summary

Msummary: org.apache.spark.ml.regression.LinearRegressionTrainingSummary = org.apache.spark.ml.regression.LinearRegressionTrainingSummary@ce6303a


In [15]:
Msummary.pValues

res4: Array[Double] = Array(0.0, 0.0, 0.0, 0.20711882640369517, 0.0, 0.0)


In [67]:
Msummary.predictions.show(5)

+------------------+--------------------+------------------+
|             label|            features|        prediction|
+------------------+--------------------+------------------+
|1059033.5578701235|[79545.4585743167...|1223847.0427535456|
|  1505890.91484695|[79248.6424548256...|1494937.6916173412|
|1058987.9878760849|[61287.0671786567...|1253016.7460814407|
|1260616.8066294468|[63345.2400462279...|1121224.0676507396|
| 630943.4893385402|[59982.1972257080...| 845388.7662952547|
+------------------+--------------------+------------------+
only showing top 5 rows



In [16]:
Msummary.r2

res5: Double = 0.9180238195089548
